# Song Recommender

Recommend a song to the user which is similar to their taste - if the song is currently hot give another hot song, otherwise give a song which has similar audio features

In [2]:
import numpy as np
import pandas as pd
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
import spotipy
import config
import json
import pickle
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import IFrame
import random


In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

<b> Import the previously collected data

In [4]:
top_100_df = pd.read_csv('top_music_data').drop(['Unnamed: 0'], axis=1)
features_df = pd.read_csv('features_data').drop(['Unnamed: 0'], axis=1)
all_music_df = pd.read_csv('all_music_data').drop(['Unnamed: 0'], axis=1)

<b> Clean data and assign variables

In [5]:
X = features_df._get_numeric_data()
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)
X_scaled_df.drop(columns=['duration_ms'])
kmeans = KMeans(n_clusters=5, random_state=1234)
kmeans.fit(X_scaled_df)
clusters = kmeans.predict(X_scaled_df)
X["cluster"] = clusters

<b> Define song recommender function

In [6]:
def song_recommender():
    
    song_request = input("Enter a song you like: ")
    hot_list1 = list(top_100_df.song_title)
    hot_list2 = [x.lower() for x in hot_list1]
    random_song = random.choice(hot_list1)
    random_artist = top_100_df.loc[top_100_df['song_title'] == random_song, 'artist'].values[0]
    
    while hot_list1 == hot_list1:
                      
        if song_request.lower() in hot_list2:
            
            track_id = sp.search(q='track:' + random_song, type='track')['tracks']['items'][0]['id']
            frame = IFrame(src=f"https://open.spotify.com/embed/track/{track_id}",
               width="320",
               height="80",
               frameborder="0",
               allowtransparency="true",
               allow="encrypted-media",
              )
            return print("This is currently a hot song! \nHere's another hot song you might like: " + " '" + 
                         random_song + "' " + ' by ' + random_artist), display(frame)

            
        elif song_request.lower() not in hot_list2:
            
            try:
                
                user_song_id = sp.search(q='track:' + song_request, type='track')['tracks']['items'][0]['id']
                song_name = sp.track('spotify:track:' + user_song_id)['name']
                artist_name = sp.track('spotify:track:' + user_song_id)['artists'][0]['name']
                user_features = pd.DataFrame(sp.audio_features(user_song_id))
                user_X = user_features._get_numeric_data()
                user_features_scaled = pd.DataFrame(scaler.transform(user_X), columns=user_X.columns)
                cluster_number = kmeans.predict(user_features_scaled)[0]
                chosen_cluster = X[X['cluster'] == cluster_number]
                random_index = random.choice(chosen_cluster.index)
                id_X = features_df.loc[random_index]['id']
                frame = IFrame(src=f"https://open.spotify.com/embed/track/{id_X}",
                               width="320",
                               height="80",
                               frameborder="0",
                               allowtransparency="true",
                               allow="encrypted-media",
                              )
                answer = input("You chose " + " '" + song_name + "' " + " By " + artist_name + ", is this correct? Y/N: ")
                if answer.lower() in ["yes", "y"]:
                    
                    return print("Good choice! A similar song you might enjoy is: " + " '" + all_music_df.loc[all_music_df['id'] == id_X, 'song_title'].iloc[0] + "' " + 
                         " By " + all_music_df.loc[all_music_df['id'] == id_X, 'artist'].iloc[0]), display(frame)
                
                elif answer.lower() in ["no", "n"]:
                    
                    song_request = input("Sorry about that! Try another song you like: ")

            except:
        
                song_request = input("Didn't recognise that song! Try again: ")
                continue
                


# Song Recommender Demo!

In [11]:
song_recommender()

Enter a song you like: purple rain
You chose  'Purple Rain'  By Prince, is this correct? Y/N: y
Good choice! A similar song you might enjoy is:  'Chiquitita'  By ABBA


(None, None)

In [ ]:
# improvement: if wrong song chosen offer to input artist name too

In [236]:
# def hot_song_rec():
    
#     hot_list1 = list(top_100_df.song_title)
#     hot_list2 = [x.lower() for x in hot_list1]
#     random_song = random.choice(hot_list1)
#     random_artist = top_100_df.loc[top_100_df['song_title'] == random_song, 'artist'].values[0]
    
#     while hot_list1 == hot_list1:
        
#         song_request = input("Enter a song you like: ")
        
#         if song_request.lower() in hot_list2:
            
#             track_id = sp.search(q='track:' + random_song, type='track')['tracks']['items'][0]['id']
#             frame = IFrame(src=f"https://open.spotify.com/embed/track/{track_id}",
#                width="320",
#                height="80",
#                frameborder="0",
#                allowtransparency="true",
#                allow="encrypted-media",
#               )
#             return print("This is currently a hot song! \nHere's another hot song you might like: " + " '" + 
#                          random_song + "' " + ' by ' + random_artist), display(frame)

            
#         elif song_request.lower() not in hot_list2:
            
#             while song_request == song_request:
                
#                 try:

#                     user_song_id = sp.search(q='track:' + song_request, type='track')['tracks']['items'][0]['id']
#                     song_name = sp.track('spotify:track:' + user_song_id)['name']
#                     artist_name = sp.track('spotify:track:' + user_song_id)['artists'][0]['name']
#                     user_features = pd.DataFrame(sp.audio_features(user_song_id))
#                     user_X = user_features._get_numeric_data()
#                     user_features_scaled = pd.DataFrame(scaler.transform(user_X), columns=user_X.columns)
#                     cluster_number = kmeans.predict(user_features_scaled)[0]
#                     chosen_cluster = X[X['cluster'] == cluster_number]
#                     random_index = random.choice(chosen_cluster.index)
#                     id_X = features_df.loc[random_index]['id']
#                     frame = IFrame(src=f"https://open.spotify.com/embed/track/{id_X}",
#                                    width="320",
#                                    height="80",
#                                    frameborder="0",
#                                    allowtransparency="true",
#                                    allow="encrypted-media",
#                                   )
#                     answer = input("You chose " + " '" + song_name + "' " + " By " + artist_name + ", is this correct? Y/N: ")
#                     if answer.lower() in ["yes", "y"]:

#                         return print("Good choice! A similar song you might enjoy is: " + " '" + all_music_df.loc[all_music_df['id'] == id_X, 'song_title'].iloc[0] + "' " + 
#                              " By " + all_music_df.loc[all_music_df['id'] == id_X, 'artist'].iloc[0]), display(frame)

#                     elif answer.lower() in ["no", "n"]:

#                         artist1 = input("Sorry about that! Try entering the artist: ")
#                         user_song_id = sp.search(q="artist:" + artist1 + " track:" + song_request, type="track")['tracks']['items'][0]['id']
                        

#                 except:

#                     song_request = input("Didn't recognise that song! Try again: ")
#                     continue
                